In [ ]:
import pandas as pd
import pyspark.sql.functions
from pyspark.sql.functions import when
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col, lit, lower
from pyspark.sql.types import StringType,DecimalType
from pyspark.sql.functions import input_file_name, substring
import matplotlib.pyplot as plt

## Extract and Transform

In [ ]:
df = pd.read_csv('/dbfs/mnt/capstone-group2-data/datain/rawdata/indicators.csv')
df.info()

In [ ]:
## drop unnecessary columns
df = df.drop(columns=["Quartile Range","Suppression Flag","Low CI","High CI","Confidence Interval"])

In [ ]:
## drop na values
df = df.dropna()
df.info()

## Load
#### to file

In [ ]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
df = spark.createDataFrame(df) 
df.printSchema()
df.show()

In [ ]:
# Mount the capstone container
from config import storageAccount
from config import storageContainer
from config import clientSecret
from config import clientid
mount_point = "/mnt/capstone-group2-data/dataout"
    
    
configs = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs)

In [ ]:
%fs
ls /mnt/capstone-group2-data/dataout

In [ ]:
df.write.mode("overwrite").option("header", "true").csv("/mnt/capstone-group2-data/dataout/cleandata/cleanIndicators")

## Load
#### to SQL database

In [ ]:
# Mount the capstone container
from config import storageAccount
from config import storageContainer
from config import clientSecret
from config import clientid
mount_point = "/mnt/capstone-group2-data/datain"
    
    
configs = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs)

In [ ]:
df = spark.read.option("header", "true").csv("/mnt/capstone-group2-data/datain/cleandata/cleanIndicators")

In [ ]:
#Age breakout table
age_df = df[df["Group"] == "By Age"]
age_df = age_df.select("Subgroup").distinct().sort('Subgroup')
age_df = age_df.withColumnRenamed('Subgroup', 'AgeLabel')
display(age_df)

In [ ]:
#Load into SQL Database
from config import server
from config import database
from config import user
from config import password
table = "dbo.Age"

age_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
#Indicator breakout table
ind_df = df.select("Indicator").distinct()
ind_df = ind_df.withColumnRenamed('Indicator', 'IndicatorLabel')
display(ind_df)

In [ ]:
#Load into SQL Database
table = "dbo.Indicator"

ind_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
#Group breakout table
group_df = df.select("Group").distinct().sort("Group")
group_df = group_df.withColumnRenamed('Group', 'GroupLabel')
display(group_df)

In [ ]:
#Load into SQL Database
table = "dbo.[Group]"

group_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
#Education breakout table
edu_df = df[df["Group"] == "By Education"]
edu_df = edu_df.select("Subgroup").distinct().sort("Subgroup")
edu_df = edu_df.withColumnRenamed('Subgroup', 'EducationLabel')
display(edu_df)

In [ ]:
#Load into SQL Database
table = "dbo.Education"

edu_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
#Week breakout table
week_df = df.select("Week Label").distinct().sort("Week")
week_df = week_df.withColumnRenamed('Week Label', 'WeekLabel')
display(week_df)

In [ ]:
#Load into SQL Database
table = "dbo.Week"

week_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
#State breakout table
state_df = df[df["Group"] == "By State"]
state_df = state_df.select("Subgroup").distinct().sort("Subgroup")
state_df = state_df.withColumnRenamed('Subgroup', 'StateLabel')
display(state_df)

In [ ]:
#Load into SQL Database
table = "dbo.State"

state_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
race_df = df[df["Group"] == "By Race/Hispanic ethnicity"]
race_df = race_df.select("Subgroup").distinct().sort("Subgroup")
race_df = race_df.withColumnRenamed('Subgroup', 'RaceLabel')
allrace = [["All races"]]
allrace_df = spark.createDataFrame(allrace)
race_df = race_df.union(allrace_df)

race_df = race_df.withColumn(
    "RaceLabel", when(col("RaceLabel") == "Hispanic or Latino","Hispanic or Latino, any race").otherwise(col("RaceLabel"))
)

display(race_df)

In [ ]:
#Load into SQL Database
table = "dbo.Race"

race_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

Making the InsuranceIndicators table with joins

In [ ]:
df = df.drop("State")
display(df)

In [ ]:
df = df.withColumn("Sex", when((df.Group == "By Gender"),df.Subgroup))
df = df.withColumn("Sex", lower(col("Sex")))
df = df.withColumn("Race", when((df.Group == "By Race/Hispanic ethnicity"),df.Subgroup))
df = df.withColumn("Age", when((df.Group == "By Age"),df.Subgroup))
df = df.withColumn("Education", when((df.Group == "By Education"),df.Subgroup))
df = df.withColumn("State", when((df.Group == "By State"),df.Subgroup))
display(df)

In [ ]:
table = "dbo.Sex"

#Read from server
sex_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
sex_df.show()

In [ ]:
df = df.join(sex_df, df.Sex ==  sex_df.SexLabel, "left")

In [ ]:
table = "dbo.Age"

#Read from server
age_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
age_df.show()

In [ ]:
df = df.join(age_df, df.Age ==  age_df.AgeLabel, "left")

In [ ]:
table = "dbo.[Group]"

#Read from server
group_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
group_df.show()

In [ ]:
df = df.join(group_df, df.Group ==  group_df.GroupLabel, "left")

In [ ]:
table = "dbo.Indicator"

#Read from server
indicator_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
indicator_df.show()

In [ ]:
df = df.join(indicator_df, df.Indicator ==  indicator_df.IndicatorLabel, "left")

In [ ]:
table = "dbo.Race"

#Read from server
race_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
race_df.show()

In [ ]:
df = df.join(race_df, df.Race ==  race_df.RaceLabel, "left")

In [ ]:
table = "dbo.Education"

#Read from server
education_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
education_df.show()

In [ ]:
df = df.join(education_df, df.Education ==  education_df.EducationLabel, "left")

In [ ]:
table = "dbo.State"

#Read from server
state_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
state_df.show()

In [ ]:
df = df.join(state_df, df.State ==  state_df.StateLabel, "left")

In [ ]:
table = "dbo.Week"

#Read from server
week_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
week_df.show()

In [ ]:
df = df.join(week_df, col('Week Label') ==  week_df.WeekLabel, "left")
display(df)

In [ ]:
df = df.drop("Subgroup", "Indicator", "Group", "Age", "Sex", "Race", "Education", "State", "Week", "Week Label", "IndicatorLabel", "GroupLabel", "AgeLabel", "SexLabel", "RaceLabel", "EducationLabel", "StateLabel", "WeekLabel")

In [ ]:
df = df.withColumnRenamed('Value', 'DataValue')
display(df)

In [ ]:
#Load to SQL Database
table = "dbo.InsuranceCoverage"

df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()